In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

## Achieving 5913 solutions

In [ ]:
N = 3
solution = "121"

for N in range(3,8):
    new_solution = []
    for i in range(len(solution)-N+2):
        substring = solution[i:i+(N-1)]
        if all(str(x) in substring for x in range(1,N)):
            new_solution.append(substring+str(N)+substring)
    solution = new_solution[:1]
    for i in range(len(new_solution)-1):
        x1 = new_solution[i]
        x2 = new_solution[i+1]
        for j in range(1,len(x1)+1):
            if x2[:j]==x1[-1*j:]:
                break
        solution.append(x2[j:])
    solution = "".join(solution)
    print(N,len(solution))

solution = solution.replace('1','t')
solution = solution.replace('7','1')
solution = solution.replace('t','7')
solution = solution.replace('2','t')
solution = solution.replace('6','2')
solution = solution.replace('t','6')

## Splitting 5913 solution to 120 cycles of length 53

In [ ]:
solutions = {}
sub_perms = {}
ordered_solutions = []
start_idx = 0
sol = []
for i in range(len(solution)-6):
    perm = solution[i:i+7]
    if ''.join((sorted(perm)))=='1234567':
        if start_idx==-1:
            start_idx = i
            sol = []
        sol.append(perm)
        if perm.startswith('12'):
            end_idx = i+7
            
            solutions[perm] = solution[start_idx:end_idx]
            sub_perms[perm] = sol
            print(len(solutions[perm]),end=" ")
            ordered_solutions.append(solutions[perm])
            start_idx = -1

In [ ]:
ordered_solutions[0]

In [ ]:
len(ordered_solutions),len(ordered_solutions[0])

## Modify these 120 cycles so that each of output sequence contains 2 mandatory groups

In [ ]:
modified_substring_list = []
modified_substring_category = []
modified_substring_group = []

for substring in ordered_solutions:
    modified_substring = substring[-1]+'12'+substring[:-2]+substring[-1]+substring[-2]+'12'
    modified_substring_list.append(modified_substring)
    modified_substring_category.append(substring[-1]+substring[-2])
    modified_substring_group.append(substring[:3])
    print('original:',substring)
    print('modified:',modified_substring)
    print()
modified_substring_list = np.array(modified_substring_list)
modified_substring_category = np.array(modified_substring_category)
modified_substring_group = np.array(modified_substring_group)

In [ ]:
print(len(modified_substring_list[0]))

## Merging these 120 sequences

If we observe, each of these cycles begin with 'X12' and end with 'Y12'. We merge 2 sequences left-to-right if Y of left sequence (third last symbol) is equal to X of right sequence

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3)

## Split substrings in 3 sets
df = pd.DataFrame([modified_substring_group,modified_substring_category],index=['group','category']).T.sort_values(['group','category'])
df = df.reset_index(drop=True)
df_subset = df[df.category==df.category.apply(lambda x:''.join(sorted(x)))].reset_index(drop=True)
category_encodings = {x:i for i,x in enumerate(df_subset.category.unique())}
df_subset['encoded_cat'] = df_subset.category.apply(lambda x: category_encodings[x])
for split,(train_index, test_index) in enumerate(skf.split(df_subset, df_subset.encoded_cat)):
    df_subset.loc[test_index,'split'] = split
temp_splits = df_subset.set_index(['group','category']).split.to_dict()

splits = {}
for group,category in temp_splits:
    splits[(group,category)] = int(temp_splits[(group,category)])
    category_reversed = category[-1]+category[0]
    splits[(group,category_reversed)] = int((temp_splits[(group,category)]+1)%3)
    
df_subset.split.value_counts()

In [ ]:
splitted_substrings = [[],[],[]]
splitted_categories = [[],[],[]]
category2substring = [{},{},{}]

for i in range(120):
    substring = modified_substring_list[i]
    category = modified_substring_category[i]
    group = modified_substring_group[i]
    split = int(splits[group,category])
    splitted_substrings[split].append(substring)
    splitted_categories[split].append(category)
    if category not in category2substring[split]:
        category2substring[split][category] = []
    category2substring[split][category].append(substring)

## ATSP

In [ ]:
!wget http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.7.tgz 
!tar xvfz LKH-3.0.7.tgz 
!cd LKH-3.0.7; make 

In [ ]:
def get_tsp_solution(distance_matrix,timer=30,verbose=0):
    
    # CREATE DISTANCE MATRIX
    SIZE = len(distance_matrix)
    M = distance_matrix.astype(int)
            
    # WRITE PROBLEM FILE
    f = open(f'group.par','w')
    f.write("PROBLEM_FILE = ../distances.atsp\n")
    f.write("TOUR_FILE = ../output.txt\n")
    f.write(f"OPTIMUM = {SIZE}\n")
    f.write(f"BACKTRACKING = YES\n")
    f.write(f"MAX_CANDIDATES = 7 SYMMETRIC\n")
    f.write("MOVE_TYPE = 3\n")
    f.write("PATCHING_C = 3\n")
    f.write("PATCHING_A = 2\n")
    f.write("RUNS = 1\n")
    f.write(f"TIME_LIMIT = {timer}\n") #seconds
    f.close()
    
    # WRITE PARAMETER FILE
    f = open(f'distances.atsp','w')
    f.write("NAME: distances\n")
    f.write("TYPE: ATSP\n")
    f.write("COMMENT: Asymmetric TSP\n")
    f.write(f"DIMENSION: {SIZE}\n")
    f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
    f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
    f.write("EDGE_WEIGHT_SECTION\n")
    for j in range(SIZE):
        #if j%25==0: print(j,', ',end='')
        for k in range(SIZE):
            f.write(f"{M[j,k]:2d} ") 
        f.write("\n")
    f.close()
    
    # EXECUTE TSP SOLVER
    
    if verbose==0:
        !cd LKH-3.0.7; ./LKH ../group.par >> out.txt
    else:
        !cd LKH-3.0.7; ./LKH ../group.par
    
    # READ RESULTING ORDER
    with open('output.txt') as f:
        lines = f.readlines()
    for i,ln in enumerate(lines):
        if 'TOUR_SECTION' in ln: break
    perms = [int(x[:-1]) for x in lines[i+1:-2] ]
    
    
    return perms

In [ ]:
output_strings = []
output_strings_list = [[],[],[]]
for split in range(3):
    print("Solving for split",split)
    substrings = splitted_substrings[split]
    categories = splitted_categories[split]

    distance_matrix = np.ones((len(categories),len(categories)))
    for i,cat1 in enumerate(categories):
        for j,cat2 in enumerate(categories):
            if cat1[1]==cat2[0]:
                distance_matrix[i,j] = 0
                
    solution = get_tsp_solution(distance_matrix,verbose=0)

    
    output_string = ""
    for idx in solution:
        output_strings_list[split].append(substrings[idx-1])
        if len(output_string)==0:
            output_string += substrings[idx-1]
            
        else:
            output_string += substrings[idx-1][3:]
    print("length of solution",len(output_string))
    output_strings.append(output_string[:-2])

In [ ]:
## Checking if solution is valid
import itertools
LETTERS = list(map(str,range(1,8)))
permutations = [''.join(x) for x in itertools.permutations(LETTERS)]
mandatory = [x for x in permutations if x.startswith('12')]
non_mandatory = [x for x in permutations if not x.startswith('12')]
print(len(permutations),len(mandatory),len(non_mandatory))

In [ ]:
for x in non_mandatory:
    assert x in "|".join(output_strings)

In [ ]:
len(non_mandatory)

In [ ]:
global_counter = 0 
repeated = []
for x in non_mandatory:
    counter = 0
    if x in output_strings[0]:
        counter += 1
    if x in output_strings[1]:
        counter += 1
    if x in output_strings[2]:
        counter += 1
    if counter>=2:
        global_counter += 1
        repeated.append(x)
global_counter

In [ ]:
for i in range(5):
    print(repeated[i])

In [ ]:
for i in range(3):
    for x in mandatory:
        if x not in output_strings[i]:
            output_strings[i] = output_strings[i]+x

In [ ]:
for i in range(3):
    print(len(output_strings[i]))

In [ ]:
LETTERS = [
    '🎅',  # father christmas
    '🤶',  # mother christmas
    '🦌',  # reindeer
    '🧝',  # elf
    '🎄',  # christmas tree
    '🎁',  # gift
    '🎀',  # ribbon
]

In [ ]:
for i,x in enumerate(LETTERS):
    for j in range(3):
        output_strings[j] = output_strings[j].replace(str(i+1),x)

In [ ]:
for i,x in enumerate(LETTERS):
    for j in range(3):
        output_strings[j] = output_strings[j].replace(str(i+1),x)

In [ ]:
output_strings[0]

## Wildcard

In [ ]:
## Post process
wildcard = '🌟'
output_strings_pp = []
for i in range(3):
    text = output_strings[i]
    processed = text[:-22]+wildcard+text[-20:-8]+wildcard+text[-6:]
    output_strings_pp.append(processed)

In [ ]:
for i in range(3):
    print(len(output_strings_pp[i]))

In [ ]:
output_strings_pp[0]

## Submission

In [ ]:
# WRITE SUBMISSION CSV
sub = pd.DataFrame()
sub['schedule'] = output_strings_pp
sub.to_csv('submission.csv',index=False)
sub.head()